In [1]:
import pandas as pd

# Load your file (adjust path if needed)
df = pd.read_csv("../data/allegations.csv")

# Show the column names
print("COLUMN NAMES:")
print(df.columns.tolist())

# Show the first 20 rows (only the columns we will likely use)
print("\nFIRST 20 ROWS:")
print(df.head(20))


COLUMN NAMES:
['unique_mos_id', 'first_name', 'last_name', 'command_now', 'shield_no', 'complaint_id', 'month_received', 'year_received', 'month_closed', 'year_closed', 'command_at_incident', 'rank_abbrev_incident', 'rank_abbrev_now', 'rank_now', 'rank_incident', 'mos_ethnicity', 'mos_gender', 'mos_age_incident', 'complainant_ethnicity', 'complainant_gender', 'complainant_age_incident', 'fado_type', 'allegation', 'precinct', 'contact_reason', 'outcome_description', 'board_disposition']

FIRST 20 ROWS:
    unique_mos_id first_name  last_name command_now  shield_no  complaint_id  \
0           10004   Jonathan       Ruiz     078 PCT       8409         42835   
1           10007       John      Sears     078 PCT       5952         24601   
2           10007       John      Sears     078 PCT       5952         24601   
3           10007       John      Sears     078 PCT       5952         26146   
4           10009      Noemi     Sierra     078 PCT      24058         40253   
5           1

In [2]:
# Keep just what we need
df_small = df[["year_received", "fado_type"]].copy()

# Drop missing years or categories
df_small = df_small.dropna(subset=["year_received", "fado_type"])

# Make sure year is int
df_small["year_received"] = df_small["year_received"].astype(int)

# (Optional) focus on a more modern period
df_small = df_small[df_small["year_received"] >= 2000]

# Count complaints by year and type
counts = (
    df_small
    .groupby(["year_received", "fado_type"])
    .size()
    .reset_index(name="n_complaints")
)

print(counts.head())


   year_received           fado_type  n_complaints
0           2000  Abuse of Authority           125
1           2000         Discourtesy            36
2           2000               Force            63
3           2000  Offensive Language             8
4           2001  Abuse of Authority           173


In [3]:
# Total complaints per year
totals_by_year = (
    counts
    .groupby("year_received")["n_complaints"]
    .sum()
    .reset_index(name="year_total")
)

# Merge, then compute proportion
counts_prop = counts.merge(totals_by_year, on="year_received", how="left")
counts_prop["prop_of_year"] = counts_prop["n_complaints"] / counts_prop["year_total"]

print(counts_prop.head())


   year_received           fado_type  n_complaints  year_total  prop_of_year
0           2000  Abuse of Authority           125         232      0.538793
1           2000         Discourtesy            36         232      0.155172
2           2000               Force            63         232      0.271552
3           2000  Offensive Language             8         232      0.034483
4           2001  Abuse of Authority           173         344      0.502907


In [13]:
import altair as alt

# Filter to 2005+ to emphasize rise
counts_vizA = counts[(counts["year_received"] >= 2005) & (counts["year_received"] < 2019)]

heatmap_A = (
    alt.Chart(counts_vizA)
    .mark_rect()
    .encode(
        x=alt.X("year_received:O", title="Year"),
        y=alt.Y("fado_type:N", sort=["Force", "Abuse of Authority", "Discourtesy", "Offensive Language"],
                title="Misconduct Category"),
        # color=alt.Color("n_complaints:Q",
        #                 scale=alt.Scale(scheme="reds"),  # Dramatic color scale
        #                 title="Raw Complaint Count"
        #                ),
        color=alt.Color(
            "n_complaints:Q",
            scale=alt.Scale(
                scheme="reds",
                domain=[50, 300]   # manually compress, adjust after checking min/max
            ),
            title="Raw Complaint Count"
        ),

        tooltip=["year_received", "fado_type", "n_complaints"]
    )
    .properties(
        width=650,
        height=250,
        title="Use of Force Complaints Have Surged in Recent Years"
    )
)

heatmap_A


alt.Chart(...)

In [14]:
# Start from your counts table
counts_vizA_AB = counts[
    (counts["year_received"] >= 2005) &
    (counts["year_received"] < 2019) &
    (counts["fado_type"].isin(["Force", "Discourtesy", "Offensive Language"]))
].copy()

# Check range to set color domain
counts_vizA_AB["n_complaints"].describe()


count     42.000000
mean     243.428571
std      164.333412
min       17.000000
25%       54.750000
50%      275.000000
75%      377.750000
max      502.000000
Name: n_complaints, dtype: float64

In [36]:
import altair as alt

# Filter: only AA + comparison categories, remove Force
counts_vizA_AA = counts[
    (counts["year_received"] >= 2005) &
    (counts["year_received"] < 2019) &
    (counts["fado_type"].isin(["Abuse of Authority", "Discourtesy", "Offensive Language"]))
].copy()

# Determine color limits
min_n = counts_vizA_AA["n_complaints"].min()
max_n = counts_vizA_AA["n_complaints"].max()

heatmap_AA = (
    alt.Chart(counts_vizA_AA)
    .mark_rect()
    .encode(
        x=alt.X("year_received:O", title="Year"),
        y=alt.Y(
            "fado_type:N",
            sort=["Abuse of Authority", "Discourtesy", "Offensive Language"],
            title="Misconduct Category"
        ),
        # color=alt.Color(
        #     "n_complaints:Q",
        #     scale=alt.Scale(
        #         scheme="reds",
        #         domain=[min_n, max_n]   # scaled only to AA + small cats
        #     ),
        #     title="Raw Complaint Count"
        # ),
        color=alt.Color(
            "n_complaints:Q",
            scale=alt.Scale(
                scheme="reds",
                domain=[200, max_n]   # shift lower bound upward
            )
        ),
    
        tooltip=["year_received", "fado_type", "n_complaints"]
    )
    .properties(
        width=800,
        height=330,
        title="Abuse of Authority Complaints Have Surged in Recent Years"
    )
)

heatmap_AA
# heatmap_AA.save('heatmap_abuse_authority.png')


alt.Chart(...)

In [35]:
import altair as alt

# Filter to AA + comparison categories, 2005–2018
line_df = counts[
    (counts["year_received"] >= 2005) &
    (counts["year_received"] < 2015) &
    (counts["fado_type"].isin(["Abuse of Authority", "Discourtesy", "Offensive Language"]))
].copy()

base = alt.Chart(line_df).encode(
    x=alt.X("year_received:O", title="Year"),
    y=alt.Y("n_complaints:Q", title="Complaint Count"),
)



line_chart = (
    # include AA line 
    alt.Chart(line_df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year_received:O", title="Year"),
        y=alt.Y("n_complaints:Q", title="Complaint Count"),
        color=alt.Color("fado_type:N", title="Category",
                        scale=alt.Scale(scheme="dark2")),
        tooltip=["year_received", "fado_type", "n_complaints"]
    )
    .properties(
        width=650,
        height=350,
        title="Abuse of Authority Complaints Have Risen Compared to Other Categories"
    ) 
)

line_chart


alt.Chart(...)

In [31]:
import altair as alt

# Filter to AA + comparison categories, 2005–2014 (your current range)
line_df = counts[
    (counts["year_received"] >= 2005) &
    (counts["year_received"] < 2015) &
    (counts["fado_type"].isin(["Abuse of Authority", "Discourtesy", "Offensive Language"]))
].copy()

# Base chart (your original chart)
base_chart = (
    alt.Chart(line_df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year_received:O", title="Year"),
        y=alt.Y("n_complaints:Q", title="Complaint Count"),
        color=alt.Color("fado_type:N", title="Category",
                        scale=alt.Scale(scheme="dark2")),
        tooltip=["year_received", "fado_type", "n_complaints"]
    )
)

# Overlay AA line ONLY, with thicker darker styling
aa_overlay = (
    alt.Chart(line_df[line_df["fado_type"] == "Abuse of Authority"])
    .mark_line(point=True, strokeWidth=4, color="black")   # emphasized AA
    .encode(
        x="year_received:O",
        y="n_complaints:Q"
    )
)

# Combine them
line_chart = (
    (base_chart + aa_overlay)
    .properties(
        width=650,
        height=350,
        title="Abuse of Authority Complaints Have Risen Compared to Other Categories"
    )
)

line_chart


alt.LayerChart(...)

In [11]:
counts_prop_filtered = counts_prop[counts_prop["year_received"] < 2019]

heatmap_B = (
    alt.Chart(counts_prop_filtered)
    .mark_rect()
    .encode(
        x=alt.X("year_received:O", title="Year"),
        y=alt.Y("fado_type:N", title="Misconduct Category"),  # sort removed
        color=alt.Color(
            "prop_of_year:Q",
            scale=alt.Scale(scheme="blues"),
            title="Proportion of Yearly Complaints"
        ),
        tooltip=["year_received", "fado_type", "prop_of_year"]
    )
    .properties(
        width=650,
        height=250,
        title="Share of Complaint Types Has Remained Relatively Stable Over Time"
    )
)

heatmap_B


alt.Chart(...)

In [39]:
import altair as alt

# We already have:
# counts_prop_filtered with columns:
#   year_received, fado_type, n_complaints, year_total, prop_of_year

counts_prop_filtered = counts_prop[(counts_prop["year_received"] >= 2005) & (counts_prop["year_received"] < 2019)]


smoothed = counts_prop_filtered.groupby("fado_type").apply(
    lambda g: g.assign(smoothed = g["prop_of_year"].rolling(3, min_periods=1).mean())
).reset_index(drop=True)


line_stable = (
    alt.Chart(smoothed)
    .mark_line(point=True)
    .encode(
        x=alt.X("year_received:O", title="Year"),
        y=alt.Y(
            "smoothed:Q",
            title="Proportion of Yearly Complaints",
            scale=alt.Scale(domain=[0, 0.7])  # keeps everything in a moderate range
        ),
        color=alt.Color(
            "fado_type:N",
            title="Misconduct Category"
        ),
        tooltip=["year_received", "fado_type", "prop_of_year"]
    )
    .properties(
        width=650,
        height=300,
        title="Proportion of Complaint Types Has Remained Stable Over Time"
    )
)

line_stable


/var/folders/vp/npcvxcc52xqfjgmswcdds6yc0000gn/T/ipykernel_64104/2855946730.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  smoothed = counts_prop_filtered.groupby("fado_type").apply(


alt.Chart(...)